In [1]:
import numpy
import csv
import numpy as np
from collections import defaultdict
import pickle

In [2]:
with open('data/disease_map.pkl', 'rb') as f:
    disease_map = pickle.load(f)
    
with open('data/values_per_column.pkl', 'rb') as f:
    values_per_column = pickle.load(f)

In [3]:
with open('data/stanford_blueprint_datathon_2019_data.csv') as f:
    reader = csv.reader(f)
    header = next(reader)
    
    header_map = {name: i for i, name in enumerate(header)}
    
diseases = ['chlamydia', 'gential_warts', 'gonorrhea', 'herpes', 'hpv',
            'other_std', 'parasitic', 'std_screen', 'syphilis', 'trich']

column_to_name = {}
index = 0
for i, name in enumerate(header):
    if i in values_per_column:
        column_to_name[index] = name + "|" + ''
        index += 1
        
        for val in values_per_column[i]: 
            column_to_name[index] = name + "|" + val
            index += 1
    else:
        column_to_name[index] = name
        index += 1

name_to_column = {column: index for index, column in column_to_name.items()}

In [4]:
policy_dictionary = {}

def add_to_dict(name):
    if name not in policy_dictionary:
        next_index = len(policy_dictionary)
        policy_dictionary[name] = next_index

with open('data/education_policy.csv') as f:
    reader = csv.reader(f, delimiter='\t')
    header = next(reader)

    header_map = {name: i for i, name in enumerate(header)}
    
    rows = list(reader)
    
    for row in rows:
        for h, column in zip(header[1:], row[1:]):
            if column == 'both' or column == 'std':
                add_to_dict(h + "|std")
            if column == 'both' or column == 'sex':
                add_to_dict(h + "|sex")
            if column == '1' or column == '2':
                add_to_dict(h + "|has")
            if column == '2':
                add_to_dict(h + "|focus")
    
    policy_to_name = {i: name for name, i in policy_dictionary.items()}
    policy_map = {}
    for row in rows:
        values = [0 for _ in range(len(policy_dictionary))]
        for h, column in zip(header[1:], row[1:]):
            if column == 'both' or column == 'std':
                values[policy_dictionary[h + "|std"]] = 1
            if column == 'both' or column == 'sex':
                values[policy_dictionary[h + "|sex"]] = 1
            if column == '1' or column == '2':
                values[policy_dictionary[h + "|has"]] = 1
            if column == '2':
                values[policy_dictionary[h + "|focus"]] = 1
            
            assert column in ('0', '1', '2', 'both', 'std', 'sex', '')
            
        policy_map[row[header_map['State']]] = values

In [5]:
# print('\n'.join(a for a in name_to_column.keys() if a.startswith('state')))

dc_column = name_to_column['state|Washington, DC']

In [6]:
disease_to_index = {disease: i for i, disease in enumerate(diseases)}

train_year = '2017-04-01'

target_disease = 'std_screen'

def get_x_y(year, target_disease):
    labels = np.load('data/labels/' + year + '.npy')[:, disease_to_index[target_disease]]
    data = np.load('data/rows/' + year + '.npy')
    non_nan_labels = ~np.isnan(labels) & (data[:, dc_column] == 0)
    return data[non_nan_labels, :], labels[non_nan_labels]
    
train_x, train_y = get_x_y(train_year, target_disease)

print(len(train_x))

51124


In [39]:
policy_names = []

def get_policy_x(data):
    result = np.zeros((data.shape[0], len(policy_dictionary)))
    
    for row_index in range(data.shape[0]):
        for state, state_policies in policy_map.items():
            state_index = name_to_column['state|' + state]
            if data[row_index, state_index] == 1:
                result[row_index, :] = state_policies
                break
        else:
            print('Could not find state?', state)
            print(name_to_column)
            print(1/0)
    
    final_columns = []
    for column, name in policy_to_name.items():
        if name in ('Contraception STD education|has', 'Abstinence STD education|focus', 'Statewide policy to teach|std'):
            final_columns.append(result[:, column])
            policy_names.append(name)
    
    return np.column_stack(final_columns)

final_names = []

def clean_for_policy_x(data):
    temp = data.copy()
    final_columns = []
    for name, column in name_to_column.items():
        if (
            name.startswith('age') or 
            name.startswith('gender') or 
            name.startswith('income') or 
            name.startswith('education')
        ) and name not in ('age|', 'age|35-44 years old', 
                           'gender|', 'gender|Male', 'income|', 'income|$45,000 - $49,999', 'education|', 'education|High School'):
            final_columns.append(temp[:, column])
            final_names.append(name)
            
    print(final_columns[0].shape)
    return np.column_stack(final_columns)
        
policy_x = get_policy_x(train_x)
cleaned_train_x = clean_for_policy_x(train_x)

(51124,)


In [40]:
import statsmodels.api as sm

p_train_x = np.hstack((cleaned_train_x, policy_x))

X = sm.add_constant(p_train_x)
model = sm.OLS(train_y, X)

def get_name(i):
    if i < len(final_names):
        return final_names[i]
    else:
        return 'Policy|' + policy_names[i - len(final_names)]
    
for i in range(1, len(model.exog_names)):
    model.exog_names[i] = get_name(i - 1)

results = model.fit()

In [41]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     44.21
Date:                Sun, 14 Apr 2019   Prob (F-statistic):               0.00
Time:                        06:04:23   Log-Likelihood:                -9225.6
No. Observations:               51124   AIC:                         1.855e+04
Df Residuals:                   51074   BIC:                         1.899e+04
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

In [10]:
names = ['Policy|Contraception STD education|has', 'Policy|Statewide policy to teach|std', 'Policy|Abstinence STD education|focus']

for name in names:
    index = policy_dictionary[name[7:]] + max(column_to_name.keys()) + 1 + 1
    print(name)
    print(results.bse[index])
    print(results.params[index])
    print(results.pvalues()[index])

Policy|Contraception STD education|has
0.0029681763728540513
-0.0019202307769807456


TypeError: 'numpy.ndarray' object is not callable

In [ ]:
import sklearn.linear_model

p_train_x = np.hstack((T_train_x, policy_x))

model =  sklearn.linear_model.Lasso(alpha=0.000000001)
model.fit(p_train_x, train_y)

In [ ]:
def get_name(i):
    if i in column_to_name:
        return column_to_name[i]
    else:
        return 'Policy|' + policy_to_name[i - max(column_to_name.keys()) - 1]

coefs = [(get_name(i), val) for i, val in enumerate(model.coef_) if val != 0]
coefs.sort(key=lambda x: -x[1])
print('\n'.join(str(a) for a in coefs))

In [ ]:
print(policy_to_name)